<a href="https://colab.research.google.com/github/TC2008B-Team5/Multiagent-Systems-T5/blob/main/MESA_Team5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U 'mesa'

In [ ]:
!pip install seaborn; solara; matplotlib; ipywidgets

                                                                                
 Usage: solara [OPTIONS] COMMAND [ARGS]...                                      
                                                                                
╭─ Options ────────────────────────────────────────────────────────────────────╮
│ --help      Show this message and exit.                                      │
╰──────────────────────────────────────────────────────────────────────────────╯
╭─ Commands ───────────────────────────────────────────────────────────────────╮
│ create            Quickly create a solara script or project.                 │
│ deploy                                                                       │
│ run               Run a Solara app.                                          │
│ search            Build search index based on ssg output                     │
│ ssg               Static site generation                                     │
│ staticbuild       Experime

In [ ]:
# Import Mesa modules
from mesa import Agent, Model
from mesa.agent import AgentSet
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector

# Import visualization modules
from mesa.visualization import SolaraViz, make_space_component
import solara

In [ ]:
class CarAgent(Agent):
    """An agent representing a car moving from a start to a destination."""

    def __init__(self, start_pos, destination_pos, model):
        super().__init__(model)
        self.start_pos = start_pos
        self.destination_pos = destination_pos
        self.path = []
        self.calculate_path()

    def calculate_path(self):
        """Calculate a simple path from start to destination."""
        current_x, current_y = self.start_pos
        dest_x, dest_y = self.destination_pos
        self.path = []

        # Move horizontally towards the destination
        step_x = 1 if dest_x > current_x else -1
        for x in range(current_x, dest_x, step_x):
            self.path.append((x + step_x, current_y))

        # Move vertically towards the destination
        step_y = 1 if dest_y > current_y else -1
        for y in range(current_y, dest_y, step_y):
            self.path.append((dest_x, y + step_y))

    def step(self):
        """Advance the agent by one step along its path."""
        if self.path:
            next_pos = self.path.pop(0)
            if self.model.grid.out_of_bounds(next_pos):
                return
            self.model.grid.move_agent(self, next_pos)
        else:
            # Keep the agent at the destination
            self.active = False

In [ ]:
class CityModel(Model):
    """A simple model of cars moving in a city grid."""

    def __init__(self, num_cars=5, width=10, height=10):
        super().__init__()
        self.num_agents = num_cars
        self.grid = MultiGrid(width, height, torus=False)
        self.agent_set = AgentSet([], random=self.random)

        # Define parking lot positions
        self.parking_lot_positions = [(0, 0), (width - 1, height - 1)]

        # Create car agents
        for _ in range(self.num_agents):
            start_pos = self.random.choice(self.parking_lot_positions)
            dest_pos = self.random.choice(
                [pos for pos in self.parking_lot_positions if pos != start_pos]
            )
            car_agent = CarAgent(start_pos, dest_pos, self)
            self.agent_set.add(car_agent)
            self.grid.place_agent(car_agent, start_pos)

        # Data collector to track the number of agents remaining in the model
        self.datacollector = DataCollector(
            model_reporters={"RemainingAgents": lambda m: len(m.agent_set)},
        )

    def step(self):
        """Advance the model by one step."""
        self.agent_set.do(lambda agent: agent.step())
        self.datacollector.collect(self)

In [ ]:
def agent_portrayal(agent):
    portrayal = {
        "x": agent.pos[0],
        "y": agent.pos[1],
        "color": "blue",
        "marker": "o",
        "size": 5,
        "alpha": 1.0,
    }
    return portrayal

In [ ]:
# Create the SpaceComponent for visualization
SpaceComponent = make_space_component(agent_portrayal)

# Create an instance of the model
model = CityModel(num_cars=5, width=10, height=10)



In [ ]:
# Create the SolaraViz page for visualization
page = SolaraViz(
    model,
    components=[SpaceComponent],
    name="Simple City Model",
)

# Display the page (in a Jupyter Notebook or Solara app)
page

Cannot show ipywidgets in text